In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import time

2024-09-17 10:20:58.704037: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 10:20:58.712038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 10:20:58.720876: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 10:20:58.723451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 10:20:58.730265: I tensorflow/core/platform/cpu_feature_guar

### GENERATE DATASET

In [11]:
def create_dataset(dataset_name, _start='2010-01-01', _end='2023-01-01'):

    stock_symbol = 'AAPL'
    data = yf.download(stock_symbol, start=_start, end=_end)

    data.rename(columns={
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Adj Close": "adjusted_close",
        "Volume": "volume",
    }, inplace=True)

    data.index.names = ['timestamp']

    # Display the data
    data.head()
    
    dataset_name = f'{dataset_name}.csv'
    data.to_csv(dataset_name)
    
    return dataset_name, f'{len(data)} rows'

In [12]:
create_dataset('historical', '2010-01-01', '2022-01-01')

[*********************100%***********************]  1 of 1 completed


('historical.csv', '3021 rows')

In [13]:
create_dataset('fresh', '2022-01-02', '2024-01-01')

[*********************100%***********************]  1 of 1 completed


('fresh.csv', '501 rows')

In [14]:
import csv

In [22]:
def load_csv(file_name):
    container = []
    file_path = f'{file_name}'

    with open(file_path, mode='r', newline='') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            container.append(row)

    return container

In [29]:
from datetime import datetime

In [24]:
foo = load_csv('../03_pipeline/datasets/finance_historical.csv')

In [26]:
foo[0]

{'timestamp': '2010-01-04',
 'open': '7.622499942779541',
 'high': '7.660714149475098',
 'low': '7.585000038146973',
 'close': '7.643214225769043',
 'adjusted_close': '6.454505920410156',
 'volume': '493729600'}

In [32]:
fobo = datetime.strptime('2010-01-04', '%Y-%m-%d')

In [33]:
int(time.mktime(fobo.timetuple()))

1262556000

In [38]:
int(time.time())

1726558550

### LOAD THE DATASET

In [3]:
data = pd.read_csv('1726548093_finance_data.csv', index_col='timestamp')  

In [4]:
close_prices = data['close'].values.reshape(-1, 1)

In [6]:
train_size = int(len(close_prices) * 0.8)
test_size = len(close_prices) - train_size

train_data, test_data = close_prices[0:train_size], close_prices[train_size:]

In [7]:
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(len(dataset)-time_step-1):
        X.append(dataset[i:(i+time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

In [8]:
# Create training and testing datasets
time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [9]:
# Reshape input to [samples, time steps, features] for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [10]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile the model
#model.compile(optimizer='adam', loss='mean_squared_error')

# Summary of the model
#model.summary()

I0000 00:00:1726550415.569298   92398 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726550415.590243   92398 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726550415.590351   92398 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726550415.610136   92398 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

InternalError: {{function_node __wrapped__FloorMod_device_/job:localhost/replica:0/task:0/device:GPU:0}} 'cuLaunchKernel(function, gridX, gridY, gridZ, blockX, blockY, blockZ, 0, reinterpret_cast<CUstream>(stream), params, nullptr)' failed with 'CUDA_ERROR_INVALID_HANDLE' [Op:FloorMod] name: 

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.17.0'

In [ ]:
tf.config.set_visible_devices([], 'GPU')

In [16]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
dir(tf.config)

['LogicalDevice',
 'LogicalDeviceConfiguration',
 'PhysicalDevice',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'experimental',
 'experimental_connect_to_cluster',
 'experimental_connect_to_host',
 'experimental_functions_run_eagerly',
 'experimental_run_functions_eagerly',
 'functions_run_eagerly',
 'get_logical_device_configuration',
 'get_soft_device_placement',
 'get_visible_devices',
 'list_logical_devices',
 'list_physical_devices',
 'optimizer',
 'run_functions_eagerly',
 'set_logical_device_configuration',
 'set_soft_device_placement',
 'set_visible_devices',
 'threading']